# Real-time Implementation and Automation with Oanda 

--------------------------------------------------------------------------------------------------------------------

_Disclaimer: <br>
The following illustrative examples are for general information and educational purposes only. <br>
It is neither investment advice nor a recommendation to trade, invest or take whatsoever actions.<br>
The below code should only be used in combination with an Oanda Practice/Demo Account and NOT with a Live Trading Account._

------------------------------------------------------------------------------------

## Recap: Historical Data, real-time Data and Orders

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
api = tpqoa.tpqoa("oanda.cfg")

In [ ]:
api.get_instruments()

__Historical Data__

In [ ]:
api.get_history(instrument = "EUR_USD", start = "2021-03-29", end = "2021-03-31",
                granularity = "M1", price = "M", localize = False)

__Streaming ticks / real-time Data__

In [ ]:
api.stream_data("EUR_USD", stop = 20) 

__Orders & Trades__

In [ ]:
api.create_order(instrument = "EUR_USD", units = 100000)

In [ ]:
api.get_positions()

In [ ]:
api.create_order(instrument = "EUR_USD", units = -100000)

In [ ]:
api.get_account_summary()

In [ ]:
api.get_transactions()

In [ ]:
api.print_transactions()

In [ ]:
order = api.create_order(instrument = "EUR_USD", units = 100000, suppress = True, ret = True)

In [ ]:
order

In [ ]:
order["price"]

In [ ]:
order2 = api.create_order(instrument = "EUR_USD", units = -100000, suppress = True, ret = True)

In [ ]:
order2 

In [ ]:
order2["price"]

In [ ]:
float(order2["pl"])

In [ ]:
order2["time"]

In [ ]:
order2["units"]

In [ ]:
order2["id"]

## Preview: A Trader Class live in action

In [ ]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta
import time

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                break
                
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
    
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
    
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0:
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
    

__Simple Contrarian: Bar_lenght = 1min | Window = 1 (1 minute)__

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", bar_length = "1min", window = 1, units = 100000)

In [ ]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 200)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

__Simple Contrarian: Bar_lenght = 1min | Window = 60 (1 hour)__

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", bar_length = "1min", window = 60, units = 100000)

In [ ]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 200)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

## Collecting and storing real-time tick data

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
api = tpqoa.tpqoa("oanda.cfg")

In [ ]:
api.stream_data("EUR_USD", stop = 10)

In [ ]:
# two steps: 1) Retrieve tick data 2) call on_sucess() method to print tick data
api.stream_data??

In [ ]:
# called by stream_data() when new data is retrieved 
api.on_success??

In [ ]:
class CloneClass(tpqoa.tpqoa):
    
    def on_success(self, time, bid, ask):
        print("Time: {} | Bid: {} | Ask:{}".format(time, bid, ask))

In [ ]:
api = CloneClass("oanda.cfg")

In [ ]:
api.get_instruments()[:5]

In [ ]:
api.stream_data("EUR_USD", stop = 5)

## Storing and resampling real-time tick data (Part 1)

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
class GetTickData(tpqoa.tpqoa):
    
    def __init__(self, config_file):
        super().__init__(config_file)
        self.tick_data = pd.DataFrame()
    
    def on_success(self, time, bid, ask):
        print(time, bid, ask)
        df = pd.DataFrame({"bid":bid, "ask":ask, "mid":(ask + bid)/2}, 
                          index = [pd.to_datetime(time)])
        self.tick_data = self.tick_data.append(df)

In [ ]:
td = GetTickData("oanda.cfg")

In [ ]:
td.stream_data("EUR_USD", stop = 10)

In [ ]:
td.tick_data

In [ ]:
td.tick_data.resample("1s", label = "right").last().ffill() # resample to 1s bars

In [ ]:
td = GetTickData("oanda.cfg")

In [ ]:
td.stream_data("EUR_USD", stop = 100)

In [ ]:
td.tick_data.resample("10s", label = "right").last()#.ffill()

## Storing and resampling real-time tick data (Part 2)

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
class GetTickData(tpqoa.tpqoa):
    
    def __init__(self, config_file, bar_length):
        super().__init__(config_file)
        self.bar_length = bar_length
        self.tick_data = pd.DataFrame()
    
    def on_success(self, time, bid, ask):
        print(time, bid, ask)
        df = pd.DataFrame({"bid":bid, "ask":ask, "mid":(ask + bid)/2}, 
                          index = [pd.to_datetime(time)])
        self.tick_data = self.tick_data.append(df)
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]

In [ ]:
td = GetTickData("oanda.cfg", "5s")

In [ ]:
td.stream_data("EUR_USD", stop = 20)

In [ ]:
td.data

In [ ]:
td.tick_data

## Storing and resampling real-time tick data (Part 3)

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
class ConTrader(tpqoa.tpqoa): # Ultimate Goal: Implementing a simple Contrarian Strategy
    
    def __init__(self, config_file, instrument, bar_length):
        super().__init__(config_file)
        self.instrument = instrument # define instrument
        self.bar_length = bar_length
        self.tick_data = pd.DataFrame()
    
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ") # Print running Tick number
        
        # collect and store tick data
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [pd.to_datetime(time)]) # mid price only
        self.tick_data = self.tick_data.append(df)
        
        self.resample_and_join() # NEW
        
    def resample_and_join(self): # NEW
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]

In [ ]:
for i in range(10):
    print(i, end = " ")

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "5s")

In [ ]:
trader.stream_data("EUR_USD", stop = 20)

In [ ]:
trader.data

In [ ]:
trader.tick_data

## Storing and resampling real-time tick data (Part 4)

__When should we resample?__ -> Only when a __full new bar__ is available!<br>
-> Whenever there are more than 5s (__bar_length__) between the __most recent tick__ and __last full bar__. 

In [ ]:
import pandas as pd
import tpqoa
from datetime import datetime

In [ ]:
class ConTrader(tpqoa.tpqoa):
    
    def __init__(self, config_file, instrument, bar_length):
        super().__init__(config_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) # Pandas Timedelta Object
        self.tick_data = pd.DataFrame()
        self.last_bar = pd.to_datetime(datetime.utcnow()).tz_localize("UTC") # UTC time at instantiation
    
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time) # Pandas Timestamp Object
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
    
    def resample_and_join(self):
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]
        self.last_bar = self.data.index[-1]  # update time of last full bar
    

In [ ]:
datetime.utcnow()

In [ ]:
pd.to_datetime(datetime.utcnow()).tz_localize("UTC")

In [ ]:
pd.to_timedelta("5s")

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "5s")
trader.stream_data(trader.instrument, stop = 20)

In [ ]:
trader.data

In [ ]:
trader.tick_data

## Storing and resampling real-time tick data (Part 5)

In [ ]:
import pandas as pd
import tpqoa
from datetime import datetime

Goal: Once we have added a new full bar...<br>
- delete the correpsonding ticks from self.tick_data
- only keep the latest tick (next bar)

(Recap: receiving the first tick of the next bar is the signal to resample)

In [ ]:
class ConTrader(tpqoa.tpqoa):
    
    def __init__(self, config_file, instrument, bar_length):
        super().__init__(config_file)
        self.instrument = instrument 
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.data = pd.DataFrame() # NEW
        self.last_bar = pd.to_datetime(datetime.utcnow()).tz_localize("UTC")
    
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ") 
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick]) 
        self.tick_data = self.tick_data.append(df)
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
    
    def resample_and_join(self):
        #self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]
        # NEW: append the most recent ticks (resampled) to self.data
        self.data = self.data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.tick_data = self.tick_data.iloc[-1:] # NEW: only keep the latest tick (next bar)
        self.last_bar = self.data.index[-1] 

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "5s")
trader.stream_data(trader.instrument, stop = 20)

In [ ]:
trader.data

In [ ]:
trader.tick_data # only most recent ticks since last full bar

## Working with historical data and real-time tick data (Part 1)

In [ ]:
import pandas as pd
import tpqoa
from datetime import datetime, timedelta

In [ ]:
now = datetime.utcnow()
now

In [ ]:
now = now - timedelta(microseconds = now.microsecond)
now

In [ ]:
yesterday = now - timedelta(days = 1)
yesterday

In [ ]:
api = tpqoa.tpqoa("oanda.cfg")

In [ ]:
instrument = "EUR_USD"

In [ ]:
df = api.get_history(instrument = instrument, start = yesterday, end = now,
                     granularity = "S5", price = "M", localize = False)
df

In [ ]:
df = df.c.dropna().to_frame()
df

In [ ]:
df.rename(columns = {"c":instrument}, inplace = True)
df

In [ ]:
df = df.resample("1min", label = "right").last().dropna().iloc[:-1]
df

In [ ]:
last_bar = df.index[-1]
last_bar

## Working with historical data and real-time tick data (Part 2)

In [ ]:
import pandas as pd
import tpqoa
from datetime import datetime, timedelta

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.data = None # first defined in get_most_recent()
        self.last_bar = None # first defined in get_most_recent()
    
    def get_most_recent(self, days = 5): # NEW
        now = datetime.utcnow()
        now = now - timedelta(microseconds = now.microsecond)
        past = now - timedelta(days = days)
        df = self.get_history(instrument = self.instrument, start = past, end = now,
                              granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
        df.rename(columns = {"c":self.instrument}, inplace = True)
        df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
        self.data = df.copy() # first defined
        self.last_bar = self.data.index[-1] # first defined
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            
    def resample_and_join(self):
        self.data = self.data.append(self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.data.index[-1]

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1m")

In [ ]:
print(datetime.utcnow())
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 50)

In [ ]:
trader.data#.tail(10)

In [ ]:
trader.tick_data

## Working with historical data and real-time tick data (Part 3)

In [ ]:
import pandas as pd
import tpqoa
from datetime import datetime, timedelta
import time

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.data = None 
        self.last_bar = None
        
        self.count = 0 # for demonstration purposes only
    
    def get_most_recent(self, days = 5):
        while True: # repeat until we get all historical bars
            self.count += 1 # for demonstration purposes only
            time.sleep(2)
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.data = df.copy()
            self.last_bar = self.data.index[-1]
            # accept, if less than [bar_lenght] has elapsed since the last full historical bar and now
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                break
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            
    def resample_and_join(self):
        self.data = self.data.append(self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.data.index[-1]

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1m")

In [ ]:
print(datetime.utcnow())
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 50)

In [ ]:
trader.data#.tail(10)

In [ ]:
trader.count

## Defining a Simple Contrarian Trading Strategy

In [ ]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta
import time

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None # NEW
        self.data = None 
        self.last_bar = None
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy() # raw!
            self.last_bar = self.raw_data.index[-1] # raw!
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                break
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            self.define_strategy() # Prepare Data / Strategy Features
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1]) # raw!
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1] # raw! 
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy() # self.raw_data new!
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy() # first defined here


In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1)

In [ ]:
print(datetime.utcnow())
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 100)

In [ ]:
trader.raw_data

In [ ]:
trader.data

## Placing Orders and Executing Trades

In [ ]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta
import time

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units # NEW
        self.position = 0 # NEW
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                break
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades() # NEW!
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
        
    def execute_trades(self): # NEW!
        if self.data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                print("GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                print("GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: # if position is short -> go/stay short
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                print("GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                print("GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0:  # if position is neutral -> go/stay neutral
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                print("GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                print("GOING NEUTRAL")
            self.position = 0

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1, units = 100000)

In [ ]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 100)

In [ ]:
trader.position

In [ ]:
trader.units

In [ ]:
trader.get_positions()

In [ ]:
if trader.position != 0: # if we have a final open position
    order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.position = 0

In [ ]:
trader.data

## Trade Monitoring and Reporting

In [ ]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta
import time

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = [] # NEW
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                break
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # collect and store tick data
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
        
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")  # NEW
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")  # NEW
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  # NEW
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  # NEW
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            self.position = 0
    
    def report_trade(self, order, going):  # NEW
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
    

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1, units = 100000)

In [ ]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 150)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

In [ ]:
trader.get_positions()

In [ ]:
trader.profits

In [ ]:
sum(trader.profits)

In [ ]:
trader.data.tail(10)

In [ ]:
trader.tick_data

## Trading other Strategies - Coding Challenge

In [1]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
plt.style.use("seaborn")

__Strategy 1__: SMA 50/200 (minutes) Crossover (200 ticks)

## Stop here if you don´t want to see the solution!

###############################################################

In [2]:
class SMATrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, SMA_S, SMA_L, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        #*****************add strategy-specific attributes here******************
        self.SMA_S = SMA_S
        self.SMA_L = SMA_L
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                break
                
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
    
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
    
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["SMA_S"] = df[self.instrument].rolling(self.SMA_S).mean()
        df["SMA_L"] = df[self.instrument].rolling(self.SMA_L).mean()
        df["position"] = np.where(df["SMA_S"] > df["SMA_L"], 1, -1)
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
    

In [3]:
trader = SMATrader("oanda.cfg", "EUR_USD", "1min", SMA_S = 50, SMA_L = 200, units = 100000)

In [4]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 200)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

V20ConnectionError: Connection to v20 REST server at https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles failed

In [ ]:
trader.data#.tail(20)

In [ ]:
trader.data.tail(30).plot(figsize = (12, 8), secondary_y = "position")
plt.show()

__Strategy 2__: Bollinger Bands SMA 20 (minutes) / 1 Standard Deviation (200 ticks) 

## Stop here if you don´t want to see the solution!

###############################################################

In [ ]:
class BollTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, SMA, dev, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        #*****************add strategy-specific attributes here******************
        self.SMA = SMA
        self.dev = dev
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                break
                
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
    
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
    
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["SMA"] = df[self.instrument].rolling(self.SMA).mean()
        df["Lower"] = df["SMA"] - df[self.instrument].rolling(self.SMA).std() * self.dev
        df["Upper"] = df["SMA"] + df[self.instrument].rolling(self.SMA).std() * self.dev
        df["distance"] = df[self.instrument] - df.SMA
        df["position"] = np.where(df[self.instrument] < df.Lower, 1, np.nan)
        df["position"] = np.where(df[self.instrument] > df.Upper, -1, df["position"])
        df["position"] = np.where(df.distance * df.distance.shift(1) < 0, 0, df["position"])
        df["position"] = df.position.ffill().fillna(0)
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
 

In [ ]:
trader = BollTrader("oanda.cfg", "EUR_USD", "1min", SMA = 20, dev = 1, units = 100000)

In [ ]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 200)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

In [ ]:
trader.data

In [ ]:
trader.data.tail(20)[["EUR_USD", "SMA", "Lower", "Upper"]].plot(figsize = (12, 8))
plt.show()

## Machine Learning Strategies (1) - Model Fitting

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [ ]:
data = pd.read_csv("five_minute.csv", parse_dates = ["time"], index_col = "time")

In [ ]:
data

In [ ]:
data["returns"] = np.log(data.div(data.shift(1)))

In [ ]:
data.dropna(inplace = True)

In [ ]:
data["direction"] = np.sign(data.returns)

In [ ]:
data

In [ ]:
lags = 2

In [ ]:
cols = []
for lag in range(1, lags + 1):
    col = "lag{}".format(lag)
    data[col] = data.returns.shift(lag)
    cols.append(col)
data.dropna(inplace = True)

In [ ]:
data

In [ ]:
lm = LogisticRegression(C = 1e6, max_iter = 100000, multi_class = "ovr")

In [ ]:
lm.fit(data[cols], data.direction)

In [ ]:
lm

__In-Sample "Prediction"__

In [ ]:
data["pred"] = lm.predict(data[cols])

In [ ]:
data

In [ ]:
hits = np.sign(data.direction * data.pred).value_counts()

In [ ]:
hits

In [ ]:
hit_ratio = hits[1.0] / sum(hits)
hit_ratio

In [ ]:
lm

__Saving the model__

In [ ]:
import pickle

In [ ]:
pickle.dump(lm, open("logreg.pkl", "wb"))

## Machine Learning Strategies (2) - Implementation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import tpqoa
from datetime import datetime, timedelta
import time
import pickle

In [ ]:
class MLTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, lags, model, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        #*****************add strategy-specific attributes here******************
        self.lags = lags
        self.model = model
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                break
                
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
    
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
    
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df = df.append(self.tick_data) # append latest tick (== open price of current bar)
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        cols = []
        for lag in range(1, self.lags + 1):
            col = "lag{}".format(lag)
            df[col] = df.returns.shift(lag)
            cols.append(col)
        df.dropna(inplace = True)
        df["position"] = lm.predict(df[cols])
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  

In [ ]:
lm = pickle.load(open("logreg.pkl", "rb"))

In [ ]:
lm

In [ ]:
trader = MLTrader("oanda.cfg", "EUR_USD", "5min", lags = 2, model = lm, units = 100000)

In [ ]:
trader.model

In [ ]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 500)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

In [ ]:
trader.data.tail(10)

In [ ]:
trader.tick_data

## Importing a Trader Module / Class

In [ ]:
import trader as tr

In [ ]:
trader = tr.ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1, units = 100000)
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 30)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

In [ ]:
trader.data.tail(10)

####################################################################################################

## How to Time/Schedule the End of a Trading Session (Simple Contrarian)

(you can find the corresponding Video for the following lines of code in __Section 23__: Cloud Deployment (AWS) | Scheduling Trading Sessions | Full Automation)

In [ ]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta
import time

In [ ]:
datetime.utcnow().time() # current utc time

In [ ]:
pd.to_datetime("17:30").time() # desired end of trading session (in utc time)

In [ ]:
if datetime.utcnow().time() >= pd.to_datetime("17:30").time():
    print("Stop the Trading Session!")

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                break
                
    def on_success(self, time, bid, ask):
        
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(time)
        
        # define stop
        if recent_tick.time() >= pd.to_datetime("17:30").time():
            self.stop_stream = True
        
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
    
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
    
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0:
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
    

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1, units = 100000)

In [ ]:
trader.get_most_recent()
trader.stream_data(trader.instrument) # no stop after n ticks!
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0